<a href="https://colab.research.google.com/github/on1onmangoes/Building-Generative-AI-Apps/blob/main/HF_in_Torch_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! watch nvidia-smi 

>

In [2]:
! pip install transformers
from transformers import WhisperProcessor, WhisperForConditionalGeneration
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00


In [6]:
! pip install transformers
! pip install datasets
import torch
import torch.nn as nn
import pandas as pd
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.5 MB/s eta 0:00:00


In [19]:
# Getting a small sentence transformer
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
sentence_transformer = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
batch_size = 128

# Grab some data

In [20]:
dataset_url = "https://raw.githubusercontent.com/georgian-io/Multimodal-Toolkit/master/datasets/Womens_Clothing_E-Commerce_Reviews/Womens%20Clothing%20E-Commerce%20Reviews.csv"

In [21]:
# drop Na because we don't want to waste time engineering the blanks
dataset_csv = pd.read_csv(dataset_url, index_col=[0])
dataset_csv.drop(columns = ['Clothing ID', 'Rating'], inplace = True)
print("Raw dataset size: {size}".format(size = len(dataset_csv)))
dataset_csv = dataset_csv.dropna().reset_index(drop = True)
print("None-NAN dataset size: {size}".format(size = len(dataset_csv)))

Raw dataset size: 23486
None-NAN dataset size: 19662


In [22]:
dataset_csv.head(6)

,Age,Title,Review Text,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,60,Some major design flaws,I had such high hopes for this dress and reall...,0,0,General,Dresses,Dresses
1,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",1,0,General Petite,Bottoms,Pants
2,47,Flattering shirt,This shirt is very flattering to all due to th...,1,6,General,Tops,Blouses
3,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",0,4,General,Dresses,Dresses
4,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,1,1,General Petite,Tops,Knits
5,39,"Shimmer, surprisingly goes with lots","I ordered this in carbon for store pick up, an...",1,4,General Petite,Tops,Knits


In [23]:
# Truning categorical data to one-hot is not the best practice for transformer even when they are non-ordinal, acyclic, and with low cardinality.
# We do it here for demo purpose since one-hot is faster to calculate
def one_hot_mapping(ls):
  one_hot_map = {}
  
  for idx, value in enumerate(ls):
    one_hot = [0] * len(ls)
    one_hot[idx] = 1
    one_hot_map[value] = one_hot
  
  return one_hot_map

division_name_to_one_hot = one_hot_mapping(dataset_csv["Division Name"].unique().tolist())
department_name_to_one_hot = one_hot_mapping(dataset_csv["Department Name"].unique().tolist())
class_name_to_one_hot = one_hot_mapping(dataset_csv["Class Name"].unique().tolist())

In [24]:
# pre-processing
def join_text(title, review):
  res = []
  for i, j in zip(title, review):
    res.append(i + " " + j)
  return res

titles = dataset_csv["Title"].tolist()
review = dataset_csv["Review Text"].tolist()
joined_text = join_text(titles, review)
dataset_csv["text"] = joined_text
dataset_csv["Division Name"] = dataset_csv["Division Name"].apply(lambda x : division_name_to_one_hot[x])
dataset_csv["Department Name"] = dataset_csv["Department Name"].apply(lambda x : department_name_to_one_hot[x])
dataset_csv["Class Name"] = dataset_csv["Class Name"].apply(lambda x : class_name_to_one_hot[x])
dataset_csv.drop(columns = ['Title', 'Review Text'], inplace = True)

In [ ]:
dataset_csv.head(6)

,Age,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,text
0,60,0,0,"[1, 0, 0]","[1, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Some major design flaws I had such high hopes ...
1,50,1,0,"[0, 1, 0]","[0, 1, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","My favorite buy! I love, love, love this jumps..."
2,47,1,6,"[1, 0, 0]","[0, 0, 1, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Flattering shirt This shirt is very flattering...
3,49,0,4,"[1, 0, 0]","[1, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Not for the very petite I love tracy reese dre...
4,39,1,1,"[0, 1, 0]","[0, 0, 1, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Cagrcoal shimmer fun I aded this in my basket ...
5,39,1,4,"[0, 1, 0]","[0, 0, 1, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Shimmer, surprisingly goes with lots I ordered..."


In [26]:
# tokenize and organize the dataset
def process_dataset(dataset):

  encoded_text = tokenizer(dataset["text"], padding = 'max_length', max_length = 256)
  encodings = {
        "input_ids": encoded_text["input_ids"], 
        'attention_mask': encoded_text['attention_mask'],
        "age" : dataset["Age"],
        "label" : dataset["Recommended IND"],
        "positive_feedback_count" : dataset["Positive Feedback Count"],
        "division_name" : dataset["Division Name"],
        "department_name" : dataset["Department Name"],
        "class_name" : dataset["Class Name"]
  }
  return encodings

In [29]:
def get_dataloaders(dataset, batch_size = batch_size):
    return DataLoader(dataset, shuffle = True, batch_size = batch_size)

In [30]:
# get datasets ready
transformer_dataset = Dataset.from_pandas(dataset_csv)
transformer_dataset = transformer_dataset.map(process_dataset, batched = True, remove_columns = ['text', 'Age', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'])
columns = ['age', "label", "positive_feedback_count", "division_name", "department_name", "class_name", "attention_mask", "input_ids"]
transformer_dataset.set_format(type = 'torch', columns = columns)

Map:   0%|          | 0/19662 [00:00<?, ? examples/s]

In [31]:
# Split and put to loader
transformer_split = transformer_dataset.train_test_split(test_size = 0.1)

transformer_train_set = transformer_split["train"]
transformer_val_set = transformer_split["test"]

transformer_train_loader = get_dataloaders(transformer_train_set)
transformer_val_loader = get_dataloaders(transformer_val_set)

# Build a model

In [33]:
# Getting a head
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight.data)

def get_classifier(input_size, output_size):

    mlp = nn.Sequential(
                nn.Linear(in_features = input_size, out_features = input_size, bias = True), # Dense linear transformation
                nn.BatchNorm1d(input_size),
                nn.ReLU(),
                nn.Dropout(p = 0.1, inplace = False),
                nn.Linear(in_features = input_size, out_features = input_size // 2, bias = True),
                nn.BatchNorm1d(input_size // 2),
                nn.ReLU(),
                nn.Dropout(p = 0.1, inplace = False),
                nn.Linear(in_features = input_size // 2, out_features = output_size, bias = True),
                nn.Softmax(dim = 1)
            )
    return mlp.apply(weights_init)

In [34]:
# the model for training
class transformer_model(nn.Module):
    def __init__(self, backbone, input_size, output_size = 2):
        super(transformer_model, self).__init__()

        self.input_size = input_size
        self.output_size = output_size 
        self.backbone = backbone # the transformer
        self.linear_layers = get_classifier(self.input_size,        
                                            self.output_size)

    def forward(
        self,
        input_ids = None,
        attention_mask = None,
        age = None,
        positive_feedback_count = None,
        division_name = None,
        department_name = None,
        class_name = None
    ):
        # get raw embeddings for text
        sentence_embedding = self.backbone(input_ids, attention_mask)
        
        # convert to sentence embedding via pooling
        token_embeddings = sentence_embedding[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min = 1e-9)
        final_embedding = sum_embeddings / sum_mask
        
        # concate for MLP
        mlp_input = torch.cat((final_embedding, age.view(-1, 1), positive_feedback_count.view(-1, 1), division_name, department_name, class_name), 1) 

        logits = self.linear_layers(mlp_input) 
    
        return logits

In [35]:
input_size = 2 + 3 + 6 + 20 + sentence_transformer.config.hidden_size # hidden size is the final output size for a token embedding
transformer_model = transformer_model(backbone = sentence_transformer, input_size = input_size)
transformer_model.cuda()

transformer_model(
  (backbone): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-12, elementwi

# Training Loop

In [37]:
num_of_epoch = 8

best_accuracy_transformer = 0.0
best_f1_transformer = 0.0
best_precision_transformer = 0.0
best_recall_transformer = 0.0

# put into a seperate file
learning_rate = 0.00005 # transformers work on smaller learning rate
eval_step = 100
total_steps = len(transformer_train_loader) * num_of_epoch
cur_step = 0

progress_bar = tqdm(range(total_steps))
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(transformer_model.parameters(), lr = learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)


  0%|          | 0/1112 [01:07<?, ?it/s]


In [39]:
def metric_calculation(logits, labels):
    preds = torch.argmax(logits, dim = 1).tolist()
    labels = labels.tolist()

    f1 = f1_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    accuracy = accuracy_score(labels, preds)

    return [accuracy, f1, precision, recall]

In [40]:
for epoch in range(num_of_epoch):
  print("epoch {ep}/{nep}".format(ep = epoch + 1, nep = num_of_epoch))
  for batch in transformer_train_loader:
      optimizer.zero_grad()
      # train
      transformer_model.train() # never use this if you plan to freeze parameters
      outputs = transformer_model(
                                  input_ids = batch["input_ids"].cuda(),
                                  attention_mask = batch["attention_mask"].cuda(),
                                  age = batch["age"].cuda(),
                                  positive_feedback_count = batch["positive_feedback_count"].cuda(),
                                  division_name = batch["division_name"].cuda(),
                                  department_name = batch["department_name"].cuda(),
                                  class_name = batch["class_name"].cuda())
                            
      loss = loss_fn(outputs.squeeze(), batch["label"].cuda())
      loss.backward()
      optimizer.step()
      scheduler.step()
      
      # evaluation
      if cur_step % eval_step == 0:
          transformer_model.eval()
          val_logits = torch.Tensor()
          val_labels = torch.Tensor()
          with torch.no_grad():
            for eval_batch in transformer_val_loader:
                val_outputs = transformer_model(input_ids = eval_batch["input_ids"].cuda(),
                                          attention_mask = eval_batch["attention_mask"].cuda(),
                                          age = eval_batch["age"].cuda(),
                                          positive_feedback_count = eval_batch["positive_feedback_count"].cuda(),
                                          division_name = eval_batch["division_name"].cuda(),
                                          department_name = eval_batch["department_name"].cuda(),
                                          class_name = eval_batch["class_name"].cuda()
                                            )
                val_logits = torch.cat((val_logits, val_outputs.cpu()), 0)
                val_labels = torch.cat((val_labels, eval_batch["label"]))
          val_accuracy, val_f1_score, val_precision, val_recall = metric_calculation(val_logits, val_labels)

          # update running best
          best_accuracy_transformer = max(best_accuracy_transformer, val_accuracy)
          best_f1_transformer = max(best_f1_transformer, val_f1_score)
          best_precision_transformer = max(best_precision_transformer, val_precision)
          best_recall_transformer = max(best_recall_transformer, val_recall)
      cur_step += 1
      progress_bar.update(1)

epoch 1/8



 12%|█▎        | 139/1112 [05:04<11:46,  1.38it/s]

epoch 2/8



 25%|██▌       | 278/1112 [07:18<10:04,  1.38it/s]

epoch 3/8


...


 75%|███████▌  | 834/1112 [16:27<03:21,  1.38it/s]

epoch 7/8



 88%|████████▊ | 973/1112 [18:41<01:40,  1.38it/s]

epoch 8/8



100%|██████████| 1112/1112 [21:01<00:00,  1.32it/s]

In [41]:
print("the best accuracy for transformer: {acc}".format(acc = best_accuracy_transformer))
print("the best f1 score for transformer: {f1}".format(f1 = best_f1_transformer))
print("the best precision for transformer: {prec}".format(prec = best_precision_transformer))
print("the best recall for transformer: {recall}".format(recall = best_recall_transformer))

the best accuracy for transformer: 0.9186578546009151
the best f1 score for transformer: 0.9501376567757724
the best precision for transformer: 0.9763560500695411
the best recall for transformer: 1.0


In [43]:
HUGGING_FACE_USER_NAME = "on1onmangoes"